In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-11-04"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
16200,2024-11-04,Israel Superliga,15:50,Maccabi Ironi Ramat Gan,H. Haifa,1.21,4.26,169.5,1.83,1.90,-10.5,1.98,1.77,K2UHvk3U,0.826446,0.234742,0.546448,0.526316,0.061188,154.740,64.183298,0.414782,1.2,1.643168,1.369306,146.88,1.798,0.831787,0.462618,27.0,306.084,184.952363,0.604254,0.6,1.341641,2.236068,137.60,4.010,2.849307,0.710550,-50.0,108,86,1.36,1.60,155.242,270.086,0.788547,0.026540,0.079196,-2.29,-0.458,-0.458515,0.614271,0.5,-0.114271,-2.56,-0.512,-6.367187,0.349474,0.5,0.150526
16201,2024-11-04,Europa Liga Aba,17:00,Mornar Bar,Cibona,1.66,2.03,164.5,1.86,1.80,-3.5,2.01,1.68,tdFRrhu1,0.602410,0.492611,0.537634,0.555556,0.095020,276.630,108.595240,0.392565,0.6,1.341641,2.236068,403.84,3.842,2.093435,0.544882,-69.0,245.710,195.755731,0.796694,1.2,1.643168,1.369306,103.49,3.300,3.000733,0.909313,-14.0,64,79,6.31,1.31,241.516,323.758,0.141805,0.023184,0.126474,-0.39,-0.078,-8.461538,0.427073,0.5,0.072927,-3.66,-0.732,-1.407104,0.489068,0.2,-0.289068
16202,2024-11-04,Irã Superliga,08:30,Tabiat,Petro Novin Mahshahr,1.19,4.36,149.5,1.83,1.83,-10.5,1.83,1.83,x80kD5Z7,0.840336,0.229358,0.546448,0.546448,0.069694,125.532,43.979267,0.350343,3.0,0.000000,0.000000,109.12,1.502,0.451354,0.300502,49.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,88,74,1.24,9.70,125.612,0.000,0.807758,0.000000,0.000000,2.75,0.550,0.345455,0.738337,0.9,0.161663,0.00,0.000,inf,0.000000,0.0,0.000000
16203,2024-11-04,Irã Superliga,09:30,Palayesh Naft Abadan,Payesh Part Shahrod,1.28,3.49,150.5,1.83,1.83,-8.5,1.83,1.83,8S8YGmzq,0.781250,0.286533,0.546448,0.546448,0.067783,178.678,67.752395,0.379187,1.8,1.643168,0.912871,123.19,2.210,1.334223,0.603721,47.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,97,0,1.27,0.00,148.874,0.000,0.655223,0.000000,0.000000,-0.10,-0.020,-14.000000,0.570155,0.6,0.029845,0.00,0.000,inf,0.000000,0.0,0.000000
16204,2024-11-04,Israel Superliga,13:40,Ironi Kiryat Ata,Bnei Herzliya,1.78,1.97,166.5,1.85,1.89,-2.5,2.00,1.70,v71reAmB,0.561798,0.507614,0.540541,0.529101,0.069412,355.364,230.261164,0.647959,1.2,1.643168,1.369306,227.00,4.344,2.696475,0.620736,-6.0,198.170,149.380009,0.753797,1.2,1.643168,1.369306,461.36,2.430,1.930919,0.794617,-13.0,100,79,2.27,5.84,223.606,328.246,0.071653,0.015125,0.114666,3.69,0.738,1.056911,0.400130,0.6,0.199870,-5.00,-1.000,-0.970000,0.390967,0.3,-0.090967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16324,2024-11-04,Eua Ncaa,21:00,Georgia Southern,Northern Illinois,1.25,4.19,147.5,1.80,1.95,-9.5,2.00,1.71,lO1nf95L,0.800000,0.238663,0.555556,0.512821,0.038663,291.978,183.726882,0.629249,1.8,1.643168,0.912871,529.60,3.686,2.463824,0.668428,25.0,405.048,341.838215,0.843945,1.2,1.643168,1.369306,148.20,5.652,4.461784,0.789417,-21.0,80,78,6.62,1.90,199.956,426.728,0.764299,0.056569,0.110545,0.11,0.022,11.363636,0.478746,0.6,0.121254,8.40,1.680,1.898810,0.320105,0.3,-0.020105
16325,2024-11-04,Eua Ncaa,21:00,Florida State,Northern Kentucky,1.26,4.14,146.5,1.91,1.91,-8.5,1.95,1.77,Odl99oai,0.793651,0.241546,0.523560,0.523560,0.035

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
16255,21:00,Eua Ncaa,Loyola MD,Columbia,2.74,Back Home
16259,20:30,Eua Ncaa,Delaware,Bucknell,1.58,Back Home
16267,19:00,Eua Ncaa,Harvard,Marist,1.58,Back Home
16269,14:00,Eua Ncaa,Southern Illinois,Coll Of Charltn,2.22,Back Home
16279,23:30,Eua Nba,New Orleans Pelicans,Portland Trail Blazers,1.64,Back Home
16282,22:45,Eua Nba,Houston Rockets,New York Knicks,2.16,Back Home
16284,22:15,Eua Nba,Miami Heat,Sacramento Kings,1.69,Back Home
16285,22:00,Eua Nba,Brooklyn Nets,Memphis Grizzlies,2.69,Back Home
16297,21:00,Eua Ncaa,Cincinnati,Arkansas-Pine Bluff,1.67,Back Home
16299,22:00,Eua Ncaa,La Lafayette,Kent State,1.91,Back Home
